# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier

import pickle

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('df', engine)
# df_sample = df.sample(n=2620)
X = df['message'].values
y = df.iloc[:, 4:].values
category_names = df.iloc[:, 4:].columns

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    # normalization
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text.lower())
    
    # tokenization
    words = word_tokenize(text)
    
    # stop words
    stop_words = stopwords.words("english")
    
    # remove stop words
    words = [w for w in words if w not in stopwords.words("english")]
    
    # stemming
    stemmed = [PorterStemmer().stem(w) for w in words]
    
    # words in their root fom
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in stemmed]
    
    return lemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
# build the pipeline

pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
# separating the data using train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
# predict the values
y_pred = pipeline.predict(X_test)

In [8]:
# classification report
print(classification_report(y_test, y_pred, target_names=category_names));

                        precision    recall  f1-score   support

               related       0.85      0.92      0.88      4957
               request       0.81      0.44      0.57      1112
                 offer       0.00      0.00      0.00        27
           aid_related       0.75      0.63      0.68      2711
          medical_help       0.67      0.08      0.15       562
      medical_products       0.61      0.06      0.10       353
     search_and_rescue       0.57      0.04      0.08       191
              security       0.50      0.02      0.04       130
              military       0.66      0.14      0.23       219
           child_alone       0.00      0.00      0.00         0
                 water       0.90      0.38      0.54       413
                  food       0.86      0.50      0.63       721
               shelter       0.81      0.30      0.44       587
              clothing       0.75      0.10      0.18        89
                 money       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
# using GridSearchCV to find better parameters to use
parameters = {'clf__estimator__n_estimators': [20, 50],
                'clf__estimator__criterion' :['gini', 'entropy'],
                'clf__estimator__max_depth' : [2, 5, None],
                }

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=-1, verbose=2) 
cv.fit(X_train, y_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] clf__estimator__criterion=gini, clf__estimator__max_depth=2, clf__estimator__n_estimators=20 
[CV]  clf__estimator__criterion=gini, clf__estimator__max_depth=2, clf__estimator__n_estimators=20, total= 1.7min
[CV] clf__estimator__criterion=gini, clf__estimator__max_depth=2, clf__estimator__n_estimators=20 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  2.8min remaining:    0.0s


[CV]  clf__estimator__criterion=gini, clf__estimator__max_depth=2, clf__estimator__n_estimators=20, total= 1.7min
[CV] clf__estimator__criterion=gini, clf__estimator__max_depth=2, clf__estimator__n_estimators=20 
[CV]  clf__estimator__criterion=gini, clf__estimator__max_depth=2, clf__estimator__n_estimators=20, total= 1.7min
[CV] clf__estimator__criterion=gini, clf__estimator__max_depth=2, clf__estimator__n_estimators=50 
[CV]  clf__estimator__criterion=gini, clf__estimator__max_depth=2, clf__estimator__n_estimators=50, total= 1.8min
[CV] clf__estimator__criterion=gini, clf__estimator__max_depth=2, clf__estimator__n_estimators=50 
[CV]  clf__estimator__criterion=gini, clf__estimator__max_depth=2, clf__estimator__n_estimators=50, total= 1.8min
[CV] clf__estimator__criterion=gini, clf__estimator__max_depth=2, clf__estimator__n_estimators=50 
[CV]  clf__estimator__criterion=gini, clf__estimator__max_depth=2, clf__estimator__n_estimators=50, total= 1.8min
[CV] clf__estimator__criterion=gin

[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed: 127.8min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [20, 50], 'clf__estimator__criterion': ['gini', 'entropy'], 'clf__estimator__max_depth': [2, 5, None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [10]:
# optimal parameters
cv.best_params_

{'clf__estimator__criterion': 'gini',
 'clf__estimator__max_depth': None,
 'clf__estimator__n_estimators': 50}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
# using the optimized parameters to see if the model improved
cv_pred = cv.predict(X_test)
print(classification_report(y_test, cv_pred, target_names=category_names));

                        precision    recall  f1-score   support

               related       0.84      0.95      0.89      4957
               request       0.83      0.50      0.62      1112
                 offer       0.00      0.00      0.00        27
           aid_related       0.77      0.71      0.74      2711
          medical_help       0.58      0.06      0.11       562
      medical_products       0.82      0.09      0.16       353
     search_and_rescue       0.80      0.02      0.04       191
              security       0.50      0.02      0.03       130
              military       0.65      0.06      0.11       219
           child_alone       0.00      0.00      0.00         0
                 water       0.92      0.35      0.51       413
                  food       0.87      0.61      0.72       721
               shelter       0.84      0.37      0.52       587
              clothing       0.67      0.11      0.19        89
                 money       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Looks like we got an improvement on all metrics except precision

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [12]:
# using AdaBoostClassifier

pipeline2 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

pipeline2.fit(X_train, y_train)
y_pred_ada = pipeline2.predict(X_test)
print(classification_report(y_test, y_pred_ada, target_names=category_names))
;

                        precision    recall  f1-score   support

               related       0.80      0.97      0.88      4957
               request       0.76      0.53      0.63      1112
                 offer       0.00      0.00      0.00        27
           aid_related       0.77      0.64      0.70      2711
          medical_help       0.61      0.26      0.36       562
      medical_products       0.65      0.31      0.42       353
     search_and_rescue       0.60      0.19      0.29       191
              security       0.26      0.05      0.08       130
              military       0.54      0.27      0.36       219
           child_alone       0.00      0.00      0.00         0
                 water       0.79      0.66      0.72       413
                  food       0.80      0.70      0.74       721
               shelter       0.76      0.57      0.65       587
              clothing       0.64      0.49      0.56        89
                 money       0.56      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


''

In [14]:
# using KNeighborsClassifier

pipeline3 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer(use_idf=False)),
        ('clf', MultiOutputClassifier(KNeighborsClassifier()))
    ])

pipeline3.fit(X_train, y_train)
y_pred_kn = pipeline3.predict(X_test)
print(classification_report(y_test, y_pred_kn, target_names=category_names));

                        precision    recall  f1-score   support

               related       0.82      0.93      0.87      4957
               request       0.71      0.55      0.62      1112
                 offer       0.00      0.00      0.00        27
           aid_related       0.73      0.51      0.60      2711
          medical_help       0.62      0.09      0.16       562
      medical_products       0.62      0.11      0.19       353
     search_and_rescue       0.80      0.04      0.08       191
              security       0.00      0.00      0.00       130
              military       0.65      0.07      0.12       219
           child_alone       0.00      0.00      0.00         0
                 water       0.77      0.48      0.59       413
                  food       0.79      0.52      0.62       721
               shelter       0.76      0.29      0.42       587
              clothing       0.62      0.18      0.28        89
                 money       0.88      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


It does look lika AdaBoost is giving us the best results

### 9. Export your model as a pickle file

In [43]:
# pickle the chosen model
with open('classifier.pkl', 'wb') as f:
    pickle.dump(pipeline2, f)

In [29]:
with open('classifier', 'rb') as f:
    dm = pickle.load(f)

In [42]:
pwd

'/home/workspace'

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.